In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [21]:
fasta_path = 'GCF_000046845.1_ASM4684v1_cds_from_genomic.fna'
genbank_path = 'GCF_000046845.1_ASM4684v1_genomic.gbff'

# Our restriction enzyme is BSA-I (binds to GGTCTC/CCAGAG skips 1 bp and then cuts with an overhang of length 4).
five_p_overhang = 'GAACGGTCTCAGCAT' # GCAT (at end)
three_p_overhang = 'GTCGTGAGACCTTACG' # GTCG (at start)
print(len(five_p_overhang))

15


In [3]:
df = pd.read_csv('TPM_matrix.csv')
locus_tags = list(df.tag)
df.head()

,gene,tag,target_id,LB0R1,LB0R2,LB0R3,LB1R1,LB1R2,LB1R3,M9LQR1,M9LQR2,M9LQR3,M9SFR1,M9SFR2,M9SFR3
0,dnaA,ACIAD_RS00005,lcl|NC_005966.1_cds_WP_004930068.1_1,254.4710,195.2650,258.4730,275.958,240.0250,217.3340,56.1754,55.1445,45.2804,147.637,127.7610,157.6420
1,dnaN,ACIAD_RS00010,lcl|NC_005966.1_cds_WP_004930066.1_2,257.9470,222.5650,246.4550,334.218,211.9420,223.8630,27.4375,51.0805,53.3345,181.774,142.9680,218.1870
2,recF,ACIAD_RS00015,lcl|NC_005966.1_cds_WP_004930061.1_3,217.0360,206.5120,152.5050,141.987,180.9460,144.0180,81.6377,64.7067,70.0990,48.584,52.2221,57.3814
3,gyrB,ACIAD_RS00020,lcl|NC_005966.1_cds_WP_004930060.1_4,496.6680,476.1380,357.3170,399.349,494.2160,271.7890,84.4136,91.6070,105.7230,143.101,141.5100,151.8540
4,NaN,ACIAD_RS00025,lcl|NC_005966.1_cds_WP_004930057.1_5,47.7092,56.7863,48.9246,133.345,55.1851,44.6848,1739.3300,2147.7500,1420.1400,292.355,458.4010,280.9240


In [4]:
# get_interregions

seq_record = SeqIO.read(open(genbank_path,'r'), 'genbank')
genome_size = len(seq_record)
cds_list = []
# Loop over the gnome, get CDS features on each of the strands (locations)
for feature in seq_record.features:
    if feature.type == 'CDS': # coding sequence
        mystart = feature.location.start.position
        myend = feature.location.end.position
        strand = feature.strand
        locus_tag = feature.qualifiers['locus_tag'][0] # each cds should have only a single tag
        cds_list.append([mystart,myend,strand,locus_tag])

In [5]:
def get_fwd_start_and_end(this_start,curr_cds,strand):
    init_cds = deepcopy(curr_cds)
    for ii in range(curr_cds,-1,-1): # get nearest upstream cds on same strand
        if cds_list[ii][2] == strand:
            last_end = cds_list[ii][1]
            if curr_cds == (len(cds_list)-1) and this_start < last_end: 
                if genome_size - last_end + this_start > 40: # 40 is min promoter region length in bp
                    return this_start, last_end
            elif curr_cds != (len(cds_list)-1) and this_start - last_end > 40:
                return this_start, last_end
            else: 
                this_start = cds_list[ii][0]
                curr_cds = ii - 1
                this_start, last_end = get_fwd_start_and_end(this_start,curr_cds,strand)
                
def get_rev_start_and_end(this_start,curr_cds,strand):
    for ii in range(curr_cds,len(cds_list)): # get nearest downstream cds on same strand
        if cds_list[ii][2] == strand:
            last_end = cds_list[ii][0]
            if last_end - this_start > 40: 
                return this_start, last_end
            else: 
                this_start = cds_list[ii][1]
                curr_cds = ii + 1
                if curr_cds == len(cds_list) or ii == len(cds_list): # then we have exhausted the downstream coding sequences on this strand
                    return this_start, last_end
                else: 
                    this_start, last_end = get_rev_start_and_end(this_start,curr_cds,strand)

In [6]:
intergenic_records = []

# forward strand
for ii in range(len(cds_list)):
    strand = cds_list[ii][2]
    if strand == 1: # only look upstream for intergenic region
        
        if ii == 0: 
            this_start = cds_list[ii][0]
            this_start, last_end = get_fwd_start_and_end(this_start,len(cds_list)-1,strand)
            if this_start < last_end: 
                intergene_seq = seq_record.seq[last_end:genome_size] + seq_record.seq[0:this_start]
            else: 
                intergene_seq = seq_record.seq[last_end:this_start]
            if len(intergene_seq) > 1000: # 1000 is the maximum promoter region length in bp
                intergene_seq = intergene_seq[-1000:]
            intergene_seq = five_p_overhang + intergene_seq + three_p_overhang
            intergenic_records.append(SeqRecord(intergene_seq,id=cds_list[ii][3],
                         description='%d,%d,%d'%(last_end,this_start,strand)))  
        
        if ii > 0:
            this_start = cds_list[ii][0]
            this_start, last_end = get_fwd_start_and_end(this_start,ii-1,strand) # recursively search for intergenic region on same strand
            intergene_seq = seq_record.seq[last_end:this_start]
            if len(intergene_seq) > 1000: 
                intergene_seq = intergene_seq[-1000:]
            intergene_seq = five_p_overhang + intergene_seq + three_p_overhang
            intergenic_records.append(SeqRecord(intergene_seq,id=cds_list[ii][3],
                         description='%d,%d,%d'%(last_end,this_start,strand)))                
        

# reverse strand
for ii, features in reversed(list(enumerate(cds_list))):               
    strand = cds_list[ii][2]
    if strand == -1: # only look downstream for intergenic region
        if ii < len(cds_list)-1:
            this_start = cds_list[ii][1]
            this_start, last_end = get_rev_start_and_end(this_start,ii+1,strand)
            
            if last_end < this_start: 
                intergene_seq = seq_record.seq[this_start:genome_size] + seq_record.seq[0:last_end]
            else:
                intergene_seq = seq_record.seq[this_start:last_end]
            if len(intergene_seq) > 1000:
                intergene_seq = intergene_seq[0:1000]
            intergene_seq = five_p_overhang + intergene_seq + three_p_overhang
            intergenic_records.append(SeqRecord(intergene_seq.reverse_complement(),id=cds_list[ii][3],
                         description='%d,%d,%d'%(last_end,this_start,strand)))                


In [7]:
# place records in dataframe

intergenic_record_dict = {'locus_tag':[],'start':[],'end':[],'strand':[],'seq':[]}
for record in intergenic_records: 
    intergenic_record_dict['locus_tag'].append(record.id)
    mystart, myend, strand = record.description.split(',')
    mystart, myend, strand = int(mystart), int(myend), int(strand)
    intergenic_record_dict['start'].append(mystart)
    intergenic_record_dict['end'].append(myend)    
    intergenic_record_dict['strand'].append(strand)    
    intergenic_record_dict['seq'].append(str(record.seq))

In [8]:
intergenic_record_df = pd.DataFrame(intergenic_record_dict)
intergenic_record_df.head()
# These sequences have the GG overhangs already attached.


,locus_tag,start,end,strand,seq
0,ACIAD_RS00005,3584950,200,1,GAACGGTCTCAGCATGACGAACTTTTTTCTTGGCAACAATTACCCC...
1,ACIAD_RS00010,1598,1833,1,GAACGGTCTCAGCATTAGGTTACTCTTAAGTATATGTGTGTACTTT...
2,ACIAD_RS00015,1598,1833,1,GAACGGTCTCAGCATTAGGTTACTCTTAAGTATATGTGTGTACTTT...
3,ACIAD_RS00020,4074,4126,1,GAACGGTCTCAGCATCGTTGATTTTTTACCCATCTTTGGATAGACC...
4,ACIAD_RS00035,6595,9650,1,GAACGGTCTCAGCATAAATCACGGTCATGCGAAATCAGAACCAGTG...


In [17]:
# downselect the tags

tags_to_keep = ['ACIAD_RS00005','ACIAD_RS00010'] # placeholder for the full set 
tags_to_keep = ['ACIAD_RS00840', 'ACIAD_RS15105', 'ACIAD_RS14600', 'ACIAD_RS10530', 'ACIAD_RS16285', 
                'ACIAD_RS14395', 'ACIAD_RS06410', 'ACIAD_RS02150', 'ACIAD_RS14975', 'ACIAD_RS05830', 
                'ACIAD_RS14250', 'ACIAD_RS00535', 'ACIAD_RS14285', 'ACIAD_RS16655', 'ACIAD_RS16650', 
                'ACIAD_RS17065', 'ACIAD_RS15285', 'ACIAD_RS10135', 'ACIAD_RS16810', 'ACIAD_RS08965', 
                'ACIAD_RS15720', 'ACIAD_RS09985', 'ACIAD_RS14095', 'ACIAD_RS12890', 'ACIAD_RS08170', 
                'ACIAD_RS15350', 'ACIAD_RS15935', 'ACIAD_RS10970', 'ACIAD_RS13925', 'ACIAD_RS04970', 
                'ACIAD_RS00540', 'ACIAD_RS10095', 'ACIAD_RS14795', 'ACIAD_RS07895', 'ACIAD_RS01625', 
                'ACIAD_RS16800', 'ACIAD_RS12300', 'ACIAD_RS02525', 'ACIAD_RS07835', 'ACIAD_RS02445', 
                'ACIAD_RS16130', 'ACIAD_RS09690', 'ACIAD_RS05120', 'ACIAD_RS08850', 'ACIAD_RS16355', 
                'ACIAD_RS00570', 'ACIAD_RS17080', 'ACIAD_RS01730', 'ACIAD_RS09335', 'ACIAD_RS11675', 
                'ACIAD_RS00545', 'ACIAD_RS04270', 'ACIAD_RS16225', 'ACIAD_RS11655', 'ACIAD_RS02965', 
                'ACIAD_RS00355', 'ACIAD_RS04700', 'ACIAD_RS10955', 'ACIAD_RS11065', 'ACIAD_RS05685', 
                'ACIAD_RS04920', 'ACIAD_RS12960', 'ACIAD_RS10905', 'ACIAD_RS11455']

library_records_df = intergenic_record_df[intergenic_record_df['locus_tag'].isin(tags_to_keep)].reset_index(inplace=False)
print(library_records_df)

library_records = []
for ii in range(len(library_records_df)):
    library_records.append(SeqRecord(Seq(library_records_df.seq[ii]),id=library_records_df.locus_tag[ii],
                             description = '%d-%d,%d'%(library_records_df.start[ii],
                                                       library_records_df.end[ii],
                                                       library_records_df.strand[ii])))


    index      locus_tag    start      end  strand  \
0      36  ACIAD_RS00355    62480    68506       1   
1      66  ACIAD_RS00535   112650   115659       1   
2      67  ACIAD_RS00540   112650   115659       1   
3      68  ACIAD_RS00545   112650   115659       1   
4      73  ACIAD_RS00570   121100   121178       1   
..    ...            ...      ...      ...     ...   
58   2574  ACIAD_RS08965  1939473  1937304      -1   
59   2789  ACIAD_RS06410  1394371  1384933      -1   
60   2885  ACIAD_RS05120  1109158  1108899      -1   
61   2902  ACIAD_RS04920  1065721  1063880      -1   
62   3072  ACIAD_RS02445   527523   526685      -1   

                                                  seq  
0   GAACGGTCTCAGCATCACGTTACCTTGTACAGAGTACGGTTTACCC...  
1   GAACGGTCTCAGCATCAAAATAGTAGATTTCCTGAAAATTTAAATT...  
2   GAACGGTCTCAGCATCAAAATAGTAGATTTCCTGAAAATTTAAATT...  
3   GAACGGTCTCAGCATCAAAATAGTAGATTTCCTGAAAATTTAAATT...  
4   GAACGGTCTCAGCATCGGATTTTTAATGATCCGCCATGTGCTCTGT...  
..             

In [18]:
# write the intergenic records to a fasta file. 
# First, might want to downselect the records, which might be most easily be done by downselecting 
doWrite = True
if doWrite:
#     outpath = os.path.splitext(os.path.basename(genbank_path))[0] + "_promoters.fa"
    SeqIO.write(library_records, open('library_records.fa', 'w'), 'fasta')

In [ ]:
# intergenic_record_df.to_csv('intergenic_records.csv',index=False)